In [1]:
# matplotlib 用于绘图
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
# 处理数据的库
import numpy as np
import sklearn
import pandas as pd
# 系统库
import os
import sys
import time
# TensorFlow的库
import tensorflow as tf
from tensorflow import keras

In [2]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

In [3]:
from sklearn.model_selection import train_test_split
# test_size 指的是划分的训练集和测试集的比例
# test_size 默认值为0.25 表示数据分四份，测试集占一份
x_train_all, x_test, y_train_all, y_test = train_test_split(housing.data, housing.target, random_state = 7, test_size = 0.25)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_all, y_train_all, random_state = 11, test_size = 0.25)

In [4]:
# 数据归一化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# 训练集数据使用的是 fit_transform，和验证集与测试集中使用的 transform 是不一样的
# fit_transform 可以计算数据的均值和方差并记录下来
# 验证集和测试集用到的均值和方差都是训练集数据的，所以二者的归一化使用 transform 即可
# 归一化只针对输入数据， 标签不变
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

In [28]:
x_train_all.shape

(15480, 8)

In [26]:
x_train.shape[1:]

(8,)

In [5]:
# 函数式API实现Wide & Deep model
input = keras.layers.Input(shape=x_train.shape[1:])
# """
# 函数式API就是我们可以将模型中的层结构当做函数来用
# 如下所示就是函数式API，将input当做参数传给了hidden1
# hidden1又当做参数传递给了hidden2
# """
hidden1 = keras.layers.Dense(30, activation='relu')(input)
hidden2 = keras.layers.Dense(30, activation='relu')(hidden1)
# """
# input是wide模型的输入
# hidden2是deep模型的输出
# concatenate将二者拼接，并将拼接之后的结果传递给output layer
# """
concat = keras.layers.concatenate([input, hidden2])
output = keras.layers.Dense(1)(concat)
# """
# 固化model
# """
model = keras.models.Model(inputs = [input],
                          outputs = [output])

In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 30)           270         input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 30)           930         dense[0][0]                      
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 38)           0           input_1[0][0]                    
                                                                 dense_1[0][0]                

In [9]:
# from keras.utils import plot_model
# plot_model(model, to_file='model.png')

In [11]:
# 子类API实现Wide & Deep model
"""
子类API就是集成父类来实现
我们首先要定义一个Wide & Deep模型父类
然后通过集成该父类进行模型的构建
"""
class WideDeepModel(keras.models.Model):
    def __init__(self):
        super(WideDeepModel, self).__init__()
        """
        定义模型的层次
        """
        self.hidden1_layer = keras.layers.Dense(30, activation = 'relu')
        self.hidden2_layer = keras.layers.Dense(30, activation = 'relu')
        self.output_layer = keras.layers.Dense(1)
    
    def call(self, input):
        """
        完成模型的正向计算
        """
        hidden1 = self.hidden1_layer(input)
        hidden2 = self.hidden2_layer(hidden1)
        concat = keras.layers.concatenate([input, hidden2])
        output = self.output_layer(concat)
        return output

model = WideDeepModel()
model.build(input_shape = (None, 8))
model.summary()

Model: "wide_deep_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              multiple                  270       
_________________________________________________________________
dense_7 (Dense)              multiple                  930       
_________________________________________________________________
dense_8 (Dense)              multiple                  39        
Total params: 1,239
Trainable params: 1,239
Non-trainable params: 0
_________________________________________________________________


In [12]:
# 编译模型， 损失函数为均方误差函数，优化函数为随机梯度下降
model.compile(loss="mean_squared_error", optimizer = keras.optimizers.SGD(0.001))
# 回调函数使用了EarlyStopping，patience设为5， 阈值设置为1e-2
callbacks = [keras.callbacks.EarlyStopping(patience=5, min_delta=1e-2)]

In [35]:
x_train_scaled.shape

(11610, 8)

In [36]:
y_train.shape

(11610,)

In [13]:
history = model.fit(x_train_scaled, y_train,
                   validation_data=(x_valid_scaled, y_valid),
                   epochs = 100,
                   callbacks= callbacks)

Epoch 1/100
363/363 [==============================] - 2s 5ms/step - loss: 1.7925 - val_loss: 0.7244
Epoch 2/100
363/363 [==============================] - 1s 2ms/step - loss: 0.6265 - val_loss: 0.6484
Epoch 3/100
363/363 [==============================] - 1s 2ms/step - loss: 0.5824 - val_loss: 0.6172
Epoch 4/100
363/363 [==============================] - 1s 2ms/step - loss: 0.5570 - val_loss: 0.5951
Epoch 5/100
363/363 [==============================] - 1s 3ms/step - loss: 0.5381 - val_loss: 0.5748
Epoch 6/100
363/363 [==============================] - 1s 3ms/step - loss: 0.5231 - val_loss: 0.5562
Epoch 7/100
363/363 [==============================] - 1s 2ms/step - loss: 0.5116 - val_loss: 0.5429
Epoch 8/100
363/363 [==============================] - 1s 2ms/step - loss: 0.5014 - val_loss: 0.5311
Epoch 9/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4936 - val_loss: 0.5241
Epoch 10/100
363/363 [==============================] - 1s 4ms/step - loss: 0.4851 - val_lo

In [14]:
model.evaluate(x_test_scaled, y_test)

162/162 [==============================] - 0s 2ms/step - loss: 0.4136


0.41359058022499084

In [15]:
"""
多输入结构
"""
input_wide = keras.layers.Input(shape=[5]) # 前五个作为wide的输入
input_deep = keras.layers.Input(shape=[6]) # 后六个作为deep的输入
hidden1 = keras.layers.Dense(30, activation='relu')(input_deep)
hidden2 = keras.layers.Dense(30, activation='relu')(hidden1)
concat = keras.layers.concatenate([input_wide, hidden2])
output = keras.layers.Dense(1)(concat)

"""
固化model
"""
model = keras.models.Model(inputs = [input_wide, input_deep],
                          outputs = [output])

In [16]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 6)]          0                                            
__________________________________________________________________________________________________
dense_9 (Dense)                 (None, 30)           210         input_3[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 5)]          0                                            
__________________________________________________________________________________________________
dense_10 (Dense)                (None, 30)           930         dense_9[0][0]                    
____________________________________________________________________________________________

In [18]:
"""
x_wide:数据集前5个特征
x_deep:数据集后6个特征
"""
x_train_scaled_wide = x_train_scaled[:,:5]
x_train_scaled_deep = x_train_scaled[:,2:]
x_valid_scaled_wide = x_valid_scaled[:,:5]
x_valid_scaled_deep = x_valid_scaled[:,2:]
x_test_scaled_wide = x_test_scaled[:,:5]
x_test_scaled_deep = x_test_scaled[:,2:]

# 编译模型， 损失函数为均方误差函数，优化函数为随机梯度下降
model.compile(loss="mean_squared_error", optimizer = keras.optimizers.SGD(0.001))
# 回调函数使用了EarlyStopping，patience设为5， 阈值设置为1e-2
callbacks = [keras.callbacks.EarlyStopping(patience=5, min_delta=1e-2)]

history = model.fit([x_train_scaled_wide, x_train_scaled_deep], y_train,
                   validation_data=([x_valid_scaled_wide, x_valid_scaled_deep], y_valid),
                   epochs = 100,
                   callbacks= callbacks)

Epoch 1/100
363/363 [==============================] - 2s 3ms/step - loss: 2.1159 - val_loss: 0.8848
Epoch 2/100
363/363 [==============================] - 2s 5ms/step - loss: 0.7449 - val_loss: 0.7248
Epoch 3/100
363/363 [==============================] - 2s 6ms/step - loss: 0.6535 - val_loss: 0.6695
Epoch 4/100
363/363 [==============================] - 1s 3ms/step - loss: 0.6082 - val_loss: 0.6310
Epoch 5/100
363/363 [==============================] - 1s 2ms/step - loss: 0.5751 - val_loss: 0.6014
Epoch 6/100
363/363 [==============================] - 1s 4ms/step - loss: 0.5506 - val_loss: 0.5769
Epoch 7/100
363/363 [==============================] - 1s 3ms/step - loss: 0.5330 - val_loss: 0.5613
Epoch 8/100
363/363 [==============================] - 1s 3ms/step - loss: 0.5198 - val_loss: 0.5478
Epoch 9/100
363/363 [==============================] - 1s 3ms/step - loss: 0.5090 - val_loss: 0.5363
Epoch 10/100
363/363 [==============================] - 1s 3ms/step - loss: 0.5001 - val_lo

In [19]:
model.evaluate([x_test_scaled_wide, x_test_scaled_deep], y_test)

162/162 [==============================] - 0s 2ms/step - loss: 0.4286


0.42855796217918396

In [ ]:
https://blog.csdn.net/qq_42580947/article/details/105302840

In [31]:
class WideDeepModel(tf.keras.models.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = tf.keras.layers.Dense(30, activation='relu')
        self.hidden2 = tf.keras.layers.Dense(30, activation='relu')
        self.concat = tf.keras.layers.concatenate
        self.dense = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        input_wide, input_deep = inputs
        output_deep = self.hidden1(input_deep)
        output_deep = self.hidden2(output_deep)
        concat_input = self.concat([input_wide, output_deep])
        output = self.dense(concat_input)
        return output

    def build_graph(self, shapes):
        shape1, shape2 = shapes
        input_wide = tf.keras.layers.Input(shape=shape1)
        input_deep = tf.keras.layers.Input(shape=shape2)
        return tf.keras.models.Model(inputs=[input_wide, input_deep], outputs=[self.call([input_wide, input_deep])])

In [32]:
model = WideDeepModel()

In [33]:
# plot model
tf.keras.utils.plot_model(
    model.build_graph([(15), (15)]),
    to_file="model.png",
    show_shapes=False,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
)


('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [ ]:
# https://zhuanlan.zhihu.com/p/348968507